In [44]:
import numpy as np
import tensorflow.keras as keras
import tensorflow as tf
from importlib import reload
import InputList
import BinaryReader
import Preprocessor
import models
import Callbacks

In [210]:
reload(BinaryReader)

<module 'BinaryReader' from '/home/julius/dataspellprojects/oct-classifier/BinaryReader.py'>

## Make this such that it can be used for prediction and training pipeline

In [211]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
reader = BinaryReader.BinaryReader()  # TODO: Normalizer
training_dataset, validation_dataset = reader.create_training_datasets(InputList.training_files)
preprocesser = Preprocessor.Preprocessor(training_dataset)

Num GPUs Available:  0


In [43]:
print("Creating Normalization Layer:")
normalization_layer = preprocesser.normalize_layer()

Creating Normalization Layer:


2022-02-03 17:41:36.437851: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-02-03 17:41:36.458438: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2899885000 Hz


In [207]:
model = models.classiRaw3D(training_dataset.element_spec[0].shape, normalization_layer)

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 1536, 23, 28, 1)] 0         
_________________________________________________________________
normalization (Normalization (None, 1536, 23, 28, 1)   3         
_________________________________________________________________
permute_8 (Permute)          (None, 1, 23, 28, 1536)   0         
_________________________________________________________________
dense_8 (Dense)              (None, 1, 23, 28, 768)    1179648   
_________________________________________________________________
permute_9 (Permute)          (None, 768, 23, 28, 1)    0         
_________________________________________________________________
conv3d_16 (Conv3D)           (None, 768, 23, 28, 32)   896       
_________________________________________________________________
max_pooling3d_16 (MaxPooling (None, 384, 11, 14, 32)   0   

In [ ]:
history = model.fit(
    training_dataset,
    epochs=10,
    validation_data=Preprocessor.Preprocessor(validation_dataset).batch(20),
    callbacks=Callbacks.my_callbacks
)
model.save('savedModels/first')

In [8]:
import tensorflow as tf
tf.debugging.set_log_device_placement(True)

Num GPUs Available:  0


In [50]:
model.load_weights('checkpoints/best_model.data-00000-of-00001')

OSError: SavedModel file does not exist at: checkpoints/best_model.data-00000-of-00001/{saved_model.pbtxt|saved_model.pb}

print_function(Bag Number, dataset)

-> In einem Bag sind 7000 Instanzen

# Bag Level Model:

Ich hab ganz viele Softmaxes und von denen nehm ich einfach den Max Wert

In [58]:
test_dataset = reader.create_test_dataset([InputList.healthy_training_files[0]])
output = model.predict(test_dataset.batch(20))

KeyboardInterrupt: 